The goal of this notebook is to compute the expected image slices against which the integration tests results will be checked

In [1]:
! apt-get update -qq
! apt-get install -y -qq libgl1-mesa-glx

In [2]:
from parameterized import parameterized

import numpy as np
import torch

from diffusers import Prompt2PromptPipeline, DDIMScheduler, UNet2DModel
from diffusers.utils.testing_utils import enable_full_determinism, require_torch_gpu, slow, torch_device

np.set_printoptions(linewidth=140, precision=3)

/home/.local/share/virtualenvs/notebooks-TPEo7knB/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
enable_full_determinism()

In [4]:
replace_steps = {
    "cross_replace_steps": 0.4,
    "self_replace_steps": 0.4
}

In [5]:
test_matrix = [
    (
        ["A turtle playing with a ball", "A monkey playing with a ball"],
        "replace",
        {**replace_steps},
        [0.243, 0.233, 0.227, 0.253, 0.242, 0.237, 0.293, 0.292, 0.283]
    ), 
    (
        ["A turtle playing with a ball", "A monkey playing with a ball"],
        "replace",
        {**replace_steps, "local_blend_words": ["turtle", "monkey"]},
        [0.243, 0.233, 0.227, 0.253, 0.242, 0.237, 0.293, 0.292, 0.283]
    ), 
    (
        ["A turtle", "A turtle in a forest"],
        "refine",
        {**replace_steps},
        [0.256, 0.232, 0.209, 0.259, 0.254, 0.229, 0.285, 0.307, 0.295]
    ),
    (
        ["A turtle", "A turtle in a forest"],
        "refine",
        {**replace_steps, "local_blend_words": ["in", "a" , "forest"]},
        [0.256, 0.232, 0.209, 0.259, 0.254, 0.229, 0.285, 0.307, 0.295]
    ), 
    (
        ["A smiling turtle"] * 2,
        "reweight",
        {**replace_steps, "equalizer_words": ["smiling"], "equalizer_strengths": [5]},
        [0.006, 0.010, 0.009, 0.003, 0.011, 0.008, 0.014, 0.009, 0.000]
    ), 
]

In [6]:
from functools import partial

def expand(test, matrix): return [partial(test, *params) for params in matrix]

In [7]:
model_id = "CompVis/stable-diffusion-v1-4"

pipe = Prompt2PromptPipeline.from_pretrained(model_id)
pipe.to(torch_device)
pipe.set_progress_bar_config(disable=None)

def test_inference(prompts, edit_type, edit_kwargs, expected_slice, seed=0, n_steps=50):
    print(f"Starting next test 🎢 (prompts={prompts}, edit_type={edit_type}, edit_kwargs={edit_kwargs})")

    generator = torch.Generator().manual_seed(seed)
    image = pipe(prompts, height=512, width=512, num_inference_steps=n_steps, generator=generator, edit_type=edit_type, edit_kwargs=edit_kwargs, output_type="numpy").images

    image_slice = image[0, -3:, -3:, -1]

    assert image.shape == (2, 512, 512, 3)
    return image_slice.flatten()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.22it/s]


In [8]:
int_tests = expand(test_inference, test_matrix)

In [9]:
int_tests[0]()

Starting next test 🎢 (prompts=['A turtle playing with a ball', 'A monkey playing with a ball'], edit_type=replace, edit_kwargs={'cross_replace_steps': 0.4, 'self_replace_steps': 0.4})


/home/diffusers/diffusers/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:429: FutureWarning: The decode_latents method is deprecated and will be removed in a future version. Please use VaeImageProcessor instead
  warnings.warn(


array([0.243, 0.233, 0.227, 0.253, 0.242, 0.237, 0.293, 0.292, 0.283], dtype=float32)

Now let's run it for all test cases

In [10]:
res = []
for t in int_tests: res.append(t())

Starting next test 🎢 (prompts=['A turtle playing with a ball', 'A monkey playing with a ball'], edit_type=replace, edit_kwargs={'cross_replace_steps': 0.4, 'self_replace_steps': 0.4})
Starting next test 🎢 (prompts=['A turtle playing with a ball', 'A monkey playing with a ball'], edit_type=replace, edit_kwargs={'cross_replace_steps': 0.4, 'self_replace_steps': 0.4, 'local_blend_words': ['turtle', 'monkey']})
Starting next test 🎢 (prompts=['A turtle', 'A turtle in a forest'], edit_type=refine, edit_kwargs={'cross_replace_steps': 0.4, 'self_replace_steps': 0.4})
Starting next test 🎢 (prompts=['A turtle', 'A turtle in a forest'], edit_type=refine, edit_kwargs={'cross_replace_steps': 0.4, 'self_replace_steps': 0.4, 'local_blend_words': ['in', 'a', 'forest']})
Starting next test 🎢 (prompts=['A smiling turtle', 'A smiling turtle'], edit_type=reweight, edit_kwargs={'cross_replace_steps': 0.4, 'self_replace_steps': 0.4, 'equalizer_words': ['smiling'], 'equalizer_strengths': [5]})


In [11]:
for expected_slice in res: print(expected_slice)

[0.243 0.233 0.227 0.253 0.242 0.237 0.293 0.292 0.283]
[0.243 0.233 0.227 0.253 0.242 0.237 0.293 0.292 0.283]
[0.256 0.232 0.209 0.259 0.254 0.229 0.285 0.307 0.295]
[0.256 0.232 0.209 0.259 0.254 0.229 0.285 0.307 0.295]
[0.006 0.01  0.009 0.003 0.011 0.008 0.014 0.009 0.   ]


In [12]:
#[0.243 0.233 0.227 0.253 0.242 0.237 0.293 0.292 0.283]
#[0.243 0.233 0.227 0.253 0.242 0.237 0.293 0.292 0.283]
#[0.256 0.232 0.209 0.259 0.254 0.229 0.285 0.307 0.295]
#[0.256 0.232 0.209 0.259 0.254 0.229 0.285 0.307 0.295]
#[0.006 0.01  0.009 0.003 0.011 0.008 0.014 0.009 0.   ]


Now let's run the test with asserts

model_id = "CompVis/stable-diffusion-v1-4"

pipe = Prompt2PromptPipeline.from_pretrained(model_id)
pipe.to(torch_device)
pipe.set_progress_bar_config(disable=None)

def test_inference(prompts, edit_type, edit_kwargs, expected_slice, seed=0, n_steps=50):
    print(f"Starting next test 🎢 (prompts={prompts}, edit_type={edit_type}, edit_kwargs={edit_kwargs})")

    generator = torch.Generator().manual_seed(seed)
    image = pipe(prompts, height=512, width=512, num_inference_steps=n_steps, generator=generator, edit_type=edit_type, edit_kwargs=edit_kwargs, output_type="numpy").images

    image_slice = image[0, -3:, -3:, -1]

    assert image.shape == (2, 512, 512, 3)
    return image_slice.flatten()